1. 서울시 각 구별 cctv 수를 파악하고 인구대비 cctv 비율을 구해 순위 비교
2. 인구대비 cctv의 평균치를 확인하고 cctv가 과하게 부족한 구를 확인
3. 고령자 대비 cctv 비율
4. 외국인 대비 cctv 비율

- 사용 데이터 
- .서울시 지자체별 cctv 현황 data/01. CCTV_in_Seoul.csv
- .서울시 지자체별 인구 현황 data/01.population_in_Seoul.xls


In [1]:
# 필요 모듈 import
import pandas as pd
import numpy as np



from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"


In [2]:
import matplotlib.pyplot as plt #그래프 패키지 모듈 등록
%matplotlib inline


In [5]:
# cctv 데이터 읽어오기
# pandas의 read_csv('경로를 포함한 파일명', encoding='인코딩명')
# 데이터에 한글이 포함되어 있는 경우 : 인코딩방식을 설정
CCTV_Seoul = pd.read_csv('../data/01. CCTV_in_Seoul.csv')
CCTV_Seoul.head()
CCTV_Seoul.tail()

,기관명,소계,2013년도 이전,2014년,2015년,2016년
0,강남구,3238,1292,430,584,932
1,강동구,1010,379,99,155,377
2,강북구,831,369,120,138,204
3,강서구,911,388,258,184,81
4,관악구,2109,846,260,390,613


,기관명,소계,2013년도 이전,2014년,2015년,2016년
20,용산구,2096,1368,218,112,398
21,은평구,2108,1138,224,278,468
22,종로구,1619,464,314,211,630
23,중구,1023,413,190,72,348
24,중랑구,916,509,121,177,109


In [7]:
# CCTV_Seoul df의 컬럼명 확인
# df.columns
CCTV_Seoul.columns
CCTV_Seoul.columns[0]


Index(['기관명', '소계', '2013년도 이전', '2014년', '2015년', '2016년'], dtype='object')

'기관명'

In [8]:
# 기관명 열이름의 전달력이 떨어져서 컬럼명 변경 : 기관명 -> 구별
# 컬럼명 변경 : df.rename(columns={바꿀위치:'변경이름'}) -> 원본 반영 안됨
# df.rename(columns={바꿀위치:'변경이름'}, inplace=True) -> inplace=True : 원본변경
CCTV_Seoul.rename(columns={CCTV_Seoul.columns[0]:'구별'},inplace=True)
CCTV_Seoul.head(2)

,구별,소계,2013년도 이전,2014년,2015년,2016년
0,강남구,3238,1292,430,584,932
1,강동구,1010,379,99,155,377


In [9]:
# 서울시 인구현황 데이터 가져오기 - 엑셀파일 읽어오기
# pd.read_excel('파일명')

pop_Seoul = pd.read_excel('../data/01.population_in_Seoul.xls')
pop_Seoul.head()

# 원본데이터가 다중인덱스여서 행을 스킵해야하고 컬럼중에 필요한 컬럼만 추출할 필요가 있음



,기간,자치구,세대,인구,인구.1,인구.2,인구.3,인구.4,인구.5,인구.6,인구.7,인구.8,세대당인구,65세이상고령자
0,기간,자치구,세대,합계,합계,합계,한국인,한국인,한국인,등록외국인,등록외국인,등록외국인,세대당인구,65세이상고령자
1,기간,자치구,세대,계,남자,여자,계,남자,여자,계,남자,여자,세대당인구,65세이상고령자
2,2017.1/4,합계,4202888,10197604,5000005,5197599,9926968,4871560,5055408,270636,128445,142191,2.36,1321458
3,2017.1/4,종로구,72654,162820,79675,83145,153589,75611,77978,9231,4064,5167,2.11,25425
4,2017.1/4,중구,59481,133240,65790,67450,124312,61656,62656,8928,4134,4794,2.09,20764


In [11]:
#필요 data 만 가져오기, 자치구, 전체인구수,한국인,외국인,고령자(B,D,G,J,N)
#2행을 skip하고 3행을 header 처리

pop_Seoul = pd.read_excel('../data/01.population_in_Seoul.xls',
                         header = 2, # 행 skip
                         usecols='B,D,G,J,N') # 선택열만 가져오기
pop_Seoul.head()


,자치구,계,계.1,계.2,65세이상고령자
0,합계,10197604.0,9926968.0,270636.0,1321458.0
1,종로구,162820.0,153589.0,9231.0,25425.0
2,중구,133240.0,124312.0,8928.0,20764.0
3,용산구,244203.0,229456.0,14747.0,36231.0
4,성동구,311244.0,303380.0,7864.0,39997.0


In [14]:
# 컬럼명 변경

# 0: 구별
# 1: 인구수
# 2: 한국인
# 3: 외국인
# 4: 고령자

pop_Seoul.rename(columns={pop_Seoul.columns[0]:'구별',
                          pop_Seoul.columns[1]:'인구수',
                          pop_Seoul.columns[2]:'한국인',
                          pop_Seoul.columns[3]:'외국인',
                          pop_Seoul.columns[4]:'고령자'}, inplace=True)

pop_Seoul.head()


,구별,인구수,한국인,외국인,고령자
0,합계,10197604.0,9926968.0,270636.0,1321458.0
1,종로구,162820.0,153589.0,9231.0,25425.0
2,중구,133240.0,124312.0,8928.0,20764.0
3,용산구,244203.0,229456.0,14747.0,36231.0
4,성동구,311244.0,303380.0,7864.0,39997.0


##### 각 데이터 파악하기
- CCTV_Seoul 데이터프레임, pop_Seoul 데이터 프레임

In [17]:
#  CCTV_Seoul 데이터 프레임
CCTV_Seoul.head()
CCTV_Seoul.tail()

,구별,소계,2013년도 이전,2014년,2015년,2016년
0,강남구,3238,1292,430,584,932
1,강동구,1010,379,99,155,377
2,강북구,831,369,120,138,204
3,강서구,911,388,258,184,81
4,관악구,2109,846,260,390,613


,구별,소계,2013년도 이전,2014년,2015년,2016년
20,용산구,2096,1368,218,112,398
21,은평구,2108,1138,224,278,468
22,종로구,1619,464,314,211,630
23,중구,1023,413,190,72,348
24,중랑구,916,509,121,177,109


In [18]:
# pop_Seoul 데이터 프레임
pop_Seoul.head()
pop_Seoul.tail()

,구별,인구수,한국인,외국인,고령자
0,합계,10197604.0,9926968.0,270636.0,1321458.0
1,종로구,162820.0,153589.0,9231.0,25425.0
2,중구,133240.0,124312.0,8928.0,20764.0
3,용산구,244203.0,229456.0,14747.0,36231.0
4,성동구,311244.0,303380.0,7864.0,39997.0


,구별,인구수,한국인,외국인,고령자
22,서초구,450310.0,445994.0,4316.0,51733.0
23,강남구,570500.0,565550.0,4950.0,63167.0
24,송파구,667483.0,660584.0,6899.0,72506.0
25,강동구,453233.0,449019.0,4214.0,54622.0
26,NaN,NaN,NaN,NaN,NaN


In [19]:
# 개요 확인
CCTV_Seoul.info()
pop_Seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   구별         25 non-null     object
 1   소계         25 non-null     int64 
 2   2013년도 이전  25 non-null     int64 
 3   2014년      25 non-null     int64 
 4   2015년      25 non-null     int64 
 5   2016년      25 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 1.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   구별      26 non-null     object 
 1   인구수     26 non-null     float64
 2   한국인     26 non-null     float64
 3   외국인     26 non-null     float64
 4   고령자     26 non-null     float64
dtypes: float64(4), object(1)
memory usage: 1.2+ KB


In [20]:
# 기본 통계량 확인
CCTV_Seoul.describe()
pop_Seoul.describe()

,소계,2013년도 이전,2014년,2015년,2016년
count,25.000000,25.000000,25.000000,25.000000,25.000000
mean,1515.320000,764.760000,159.480000,205.200000,385.880000
std,616.774656,415.510257,104.132976,137.877059,182.479981
min,825.000000,238.000000,21.000000,30.000000,81.000000
25%,1010.000000,464.000000,78.000000,103.000000,292.000000
50%,1327.000000,573.000000,142.000000,184.000000,377.000000
75%,1884.000000,1070.000000,218.000000,269.000000,467.000000
max,3238.000000,1843.000000,430.000000,584.000000,932.000000


,인구수,한국인,외국인,고령자
count,2.600000e+01,2.600000e+01,26.000000,2.600000e+01
mean,7.844311e+05,7.636129e+05,20818.153846,1.016506e+05
std,1.924156e+06,1.873421e+06,51598.169516,2.491801e+05
min,1.332400e+05,1.243120e+05,2017.000000,2.076400e+04
25%,3.348055e+05,3.316718e+05,4489.000000,4.831200e+04
50%,4.135115e+05,4.051690e+05,9079.500000,5.439750e+04
75%,4.907855e+05,4.864445e+05,14901.500000,6.431075e+04
max,1.019760e+07,9.926968e+06,270636.000000,1.321458e+06


### CCTV 현황 확인

In [23]:
# 소계 컬럼을 활용해서 설치 대수가 많은 구와 적은 구를 확인(5개구를 확인)
# 소계를 기준으로 정렬 후에 head() 이용해서 확인
# 설치대수가 적은 5개 구
CCTV_Seoul.sort_values(by='소계').head()
# 도봉구, 강북구, 광진구, 강서구, 중랑구 순으로 적게 보유

,구별,소계,2013년도 이전,2014년,2015년,2016년
9,도봉구,825,238,159,42,386
2,강북구,831,369,120,138,204
5,광진구,878,573,78,53,174
3,강서구,911,388,258,184,81
24,중랑구,916,509,121,177,109


In [25]:
# 설치대수가 많은 5개 구 sort_values
CCTV_Seoul.sort_values(by='소계',ascending=False).head(5)
# 강남구가 가장 많은 CCTV 보유, 양천구, 서초구 .. 순으로 나타남

,구별,소계,2013년도 이전,2014년,2015년,2016년
0,강남구,3238,1292,430,584,932
18,양천구,2482,1843,142,30,467
14,서초구,2297,1406,157,336,398
4,관악구,2109,846,260,390,613
21,은평구,2108,1138,224,278,468


#### Data 상 최근 3개년(2014~2016) 동안 cctv가 얼마나 증가했는지 확인
- 각 연도별 cctv 대수는 해당년도에 설치한 대수를 의미

In [31]:
 
CCTV_Seoul['최근증가율']=(CCTV_Seoul['2016년']+
                          CCTV_Seoul['2015년']+
                          CCTV_Seoul['2014년'])/CCTV_Seoul['2013년도 이전'] * 100
CCTV_Seoul.head()


,구별,소계,2013년도 이전,2014년,2015년,2016년,최근증가율
0,강남구,3238,1292,430,584,932,150.619195
1,강동구,1010,379,99,155,377,166.490765
2,강북구,831,369,120,138,204,125.203252
3,강서구,911,388,258,184,81,134.793814
4,관악구,2109,846,260,390,613,149.290780


In [32]:
# 최근 증가율이 높은 3개구를 확인
CCTV_Seoul.sort_values(by='최근증가율', ascending=False).head(3)

# 도봉구는 설치된 CCTV 대수는 가장 적지만 최근 3개월 동안 증가율은 높은 편임



,구별,소계,2013년도 이전,2014년,2015년,2016년,최근증가율
22,종로구,1619,464,314,211,630,248.922414
9,도봉구,825,238,159,42,386,246.638655
12,마포구,980,314,118,169,379,212.101911


In [33]:
 # 최근 증가율이 낮은 3개구를 확인
CCTV_Seoul.sort_values(by='최근증가율', ascending=True).head(3)


,구별,소계,2013년도 이전,2014년,2015년,2016년,최근증가율
18,양천구,2482,1843,142,30,467,34.671731
13,서대문구,1254,844,50,68,292,48.578199
20,용산구,2096,1368,218,112,398,53.216374
